## Testing Julia 

In [ ]:
using LinearAlgebra

# create a simple array of 2d points
a = [(1.,2.), (3.,4.), (5.,6.)]
a = [("a", 1.1), ("b", 2.2), ("c", 3.3)]

norm_a = norm.(a)
cos_a = cos.(a)  # this will fail because cos expects a number, not a tuple

println("type of a = ", typeof(a))
println("type of norm_a = ", typeof(norm_a))
println("norm_a = ", norm_a)


In [ ]:
methods(cos)

In [ ]:
using LinearAlgebra

# now create an array of 2d vectors
a = [ [1.,2.], [3.,4.], [5.,6.]]

norm_a = norm.(a)
cos_a = cos.(a)

println("type of a = ", typeof(a))
println("type of norm_a = ", typeof(norm_a))
println("type of cos_a = ", typeof(cos_a))
println("norm_a = ", norm_a)
println("cos_a = ", cos_a)

In [ ]:
using LinearAlgebra

#create a 2d matrix
a = [1. 2.; 3. 4.; 5. 6.]

norm_a = norm.(a)
cos_a = cos.(a)

println("type of a = ", typeof(a))
println("type of norm_a = ", typeof(norm_a))
println("type of cos_a = ", typeof(cos_a))
println("norm_a = ", norm_a)
println("cos_a = ", cos_a)

In [ ]:
using LinearAlgebra

ma, mb = [1. 2. 3.; 4. 5. 6.], [7. 8. 9.; 10. 11. 12.]
va, vb = [[1.,2.,3.], [4.,5.,6.]], [[7.,8.,9.], [10.,11.,12.]]
a, b = [1., 2., 3.], [7., 8., 9.]

In [ ]:
a

In [ ]:
va

In [ ]:
map(x -> a-x, va)

In [ ]:
(a,) .- va

In [ ]:
a × b

In [ ]:
va .× vb

In [ ]:
ma[:] × mb

In [ ]:
# generate a random Float64 number from a uniform distribution between -3 and 3
rand() * 6 - 3

In [ ]:
using Distributions
# get a random number from a Uniform distribution between -3 and 3
rand(Uniform(-3, 3), 3)

In [ ]:
include("utils.jl")
V3()



## GLMakie examples

In [1]:
using GLMakie
function scatters_in_3D()
    n = 100000
    x, y, z = randn(n), randn(n), randn(n)
    aspect=(1, 1, 1)
    perspectiveness=0.7
    # the figure
    fig = Figure(; size=(1200, 400))
    ax1 = Axis3(fig[1, 1]; aspect, perspectiveness)
    ax2 = Axis3(fig[1, 2]; aspect, perspectiveness)
    ax3 = Axis3(fig[1, 3]; aspect=:data, perspectiveness)
    scatter!(ax1, x, y, z; markersize=15)
    meshscatter!(ax2, x, y, z; markersize=0.25)
    hm = meshscatter!(ax3, x, y, z; markersize=0.25,
        marker=Rect3f(Vec3f(0), Vec3f(1)), color=1:n,
        colormap=:plasma, transparency=false)
    Colorbar(fig[1, 4], hm, label="values", height=Relative(0.5))
    colgap!(fig.layout, 5)
    fig
end
scatters_in_3D()

In [2]:
using GLMakie
function lines_in_3D()
    n = 1000
    x, y, z = randn(n), randn(n), randn(n)
    aspect=(1, 1, 1)
    perspectiveness=0.5
    # the figure
    fig = Figure(; size=(1200, 500))
    ax1 = Axis3(fig[1, 1]; aspect, perspectiveness)
    ax2 = Axis3(fig[1, 2]; aspect, perspectiveness)
    ax3 = Axis3(fig[1, 3]; aspect=:data, perspectiveness)
    lines!(ax1, x, y, z; color=1:n, linewidth=3)
    scatterlines!(ax2, x, y, z; markersize=15)
    hm = meshscatter!(ax3, x, y, z; markersize=0.2, color=1:n)
    lines!(ax3, x, y, z; color=1:n)
    Colorbar(fig[2, 1], hm; label="values", height=15, vertical=false,
        flipaxis=false, ticksize=15, tickalign=1, width=Relative(3.55/4))
    fig
end
lines_in_3D()

In [3]:
using GLMakie
function peaks(; n=49)
    x = range(-3, 3, n)
    y = range(-3, 3, n)
    a = 3 * (1 .- x') .^ 2 .* exp.(-(x' .^ 2) .- (y .+ 1) .^ 2)
    b = 10 * (x' / 5 .- x' .^ 3 .- y .^ 5) .* exp.(-x' .^ 2 .- y .^ 2)
    c = 1 / 3 * exp.(-(x' .+ 1) .^ 2 .- y .^ 2)
    return (x, y, a .- b .- c)
end
function plot_peaks_function()
    x, y, z = peaks()
    x2, y2, z2 = peaks(; n=15)
    fig = Figure(size=(1200, 400))
    axs = [Axis3(fig[1, i]; aspect=(1, 1, 1)) for i = 1:3]
    hm = surface!(axs[1], x, y, z)
    wireframe!(axs[2], x2, y2, z2)
    contour3d!(axs[3], x, y, z; levels=20)
    Colorbar(fig[1, 4], hm, height=Relative(0.5))
    fig
end
plot_peaks_function()

In [4]:
using GLMakie
function heatmap_contour_and_contourf()
    x, y, z = peaks()
    fig = Figure(size=(1200, 400))
    axs = [Axis(fig[1, i]; aspect=DataAspect()) for i = 1:3]
    hm = heatmap!(axs[1], x, y, z)
    contour!(axs[2], x, y, z; levels=20)
    contourf!(axs[3], x, y, z)
    Colorbar(fig[1, 4], hm, height=Relative(0.5))
    fig
end
heatmap_contour_and_contourf()

In [5]:
using GLMakie, LinearAlgebra
function arrows_and_streamplot_in_3d()
    ps = [Point3f(x, y, z) for x=-3:1:3 for y=-3:1:3 for z=-3:1:3]
    ns = map(p -> 0.1 * rand() * Vec3f(p[2], p[3], p[1]), ps)
    lengths = norm.(ns)
    flowField(x, y, z) = Point(-y + x * (-1 + x^2 + y^2)^2,
        x + y * (-1 + x^2 + y^2)^2, z + x * (y - z^2))
    fig = Figure(size=(1200, 800), fontsize=26)
    axs = [Axis3(fig[1, i]; aspect=(1,1,1), perspectiveness=0.5) for i=1:2]
    arrows!(axs[1], ps, ns, color=lengths, arrowsize=Vec3f(0.2, 0.2, 0.3),
        linewidth=0.1)
    streamplot!(axs[2], flowField, -4 .. 4, -4 .. 4, -4 .. 4,
        colormap=:plasma, gridsize=(7, 7), arrow_size=0.25, linewidth=1)
    fig
end
arrows_and_streamplot_in_3d()

In [14]:
using GLMakie, GeometryBasics
function mesh_volume_contour()
    # mesh objects
    rectMesh = Rect3f(Vec3f(-0.5), Vec3f(1))
    recmesh = GeometryBasics.mesh(rectMesh)
    sphere = Sphere(Point3f(0), 1)
    # https://juliageometry.github.io/GeometryBasics.jl/stable/primitives/
    spheremesh = GeometryBasics.mesh(Tesselation(sphere, 64))
    # uses 64 for tesselation, a smoother sphere
    colors = [rand() for v in recmesh.position]
    # cloud points for volume
    x = y = z = 1:10
    vals = randn(10, 10, 10)
    fig = Figure(size=(1200, 400))
    axs = [Axis3(fig[1, i]; aspect=(1,1,1), perspectiveness=0.5) for i=1:3]
    mesh!(axs[1], recmesh; color=colors, colormap=:rainbow, shading=NoShading)
    mesh!(axs[1], spheremesh; color=(:white, 0.25), transparency=true)
    volume!(axs[2], x, y, z, vals; colormap=Reverse(:plasma))
    contour!(axs[3], x, y, z, vals; colormap=Reverse(:plasma))
    fig
end
mesh_volume_contour()

┌ Warning: Encountered an `AbstractVector` with value 1:10 on side x in `convert_arguments` for the `ImageLike` trait.
│         Using an `AbstractVector` to specify one dimension of an `ImageLike` is deprecated because `ImageLike` sides always need exactly two values, start and stop.
│         Use interval notation `start .. stop` or a two-element tuple `(start, stop)` instead.
└ @ Makie /Users/mg/.julia/packages/Makie/gG38B/src/conversions.jl:413
┌ Warning: Encountered an `AbstractVector` with value 1:10 on side y in `convert_arguments` for the `ImageLike` trait.
│         Using an `AbstractVector` to specify one dimension of an `ImageLike` is deprecated because `ImageLike` sides always need exactly two values, start and stop.
│         Use interval notation `start .. stop` or a two-element tuple `(start, stop)` instead.
└ @ Makie /Users/mg/.julia/packages/Makie/gG38B/src/conversions.jl:413
┌ Warning: Encountered an `AbstractVector` with value 1:10 on side z in `convert_arguments` fo

In [15]:
using GeometryBasics, Colors, GLMakie
include("utils.jl")
spheresGrid = [Point3f(i,j,k) for i in 1:2:10 for j in 1:2:10 for k in 1:2:10]
colorSphere = [RGBA(i * 0.1, j * 0.1, k * 0.1, 0.75) for i in 1:2:10 for j in 1:2:10 for k in 1:2:10]
spheresPlane = [Point3f(i,j,k) for i in 1:2.5:23 for j in 1:2.5:10 for k in 1:2.5:4]
cmap = colormap("RdBu", 50)
colorsPlane = cmap[rand(1:50,50)]
rectMesh = Rect3f(Vec3f(-1, -1, 2.1), Vec3f(16, 11, 0.5))
recmesh = GeometryBasics.mesh(rectMesh)
colors = [RGBA(rand(4)...) for v in recmesh.position]

function grid_spheres_and_rectangle_as_plate()
    perspectiveness=0.5
    aspect = :data
    # now the figure
    fig = Figure(size=MACSCREEN, theme=theme_dark())
    ax1 = Axis3(fig[1, 1]; aspect, perspectiveness, azimuth=0.72)
    ax2 = Axis3(fig[1, 2]; aspect, perspectiveness)
    meshscatter!(ax1, spheresGrid; color=colorSphere, markersize=1,
        shading=NoShading)
    meshscatter!(ax2, spheresPlane; color=colorsPlane, markersize=0.75,
        backlight=1.0f0)
    mesh!(recmesh; color=colors, colormap=:rainbow, shading=NoShading)
    limits!(ax1, 0, 10, 0, 10, 0, 10)
    fig
end
grid_spheres_and_rectangle_as_plate()

In [16]:
using GLMakie, GeometryBasics, Colors
function peaks(; n=49)
    x = range(-3, 3, n)
    y = range(-3, 3, n)
    a = 3 * (1 .- x') .^ 2 .* exp.(-(x' .^ 2) .- (y .+ 1) .^ 2)
    b = 10 * (x' / 5 .- x' .^ 3 .- y .^ 5) .* exp.(-x' .^ 2 .- y .^ 2)
    c = 1 / 3 * exp.(-(x' .+ 1) .^ 2 .- y .^ 2)
    return (x, y, a .- b .- c)
end
x, y, z = peaks(; n=15)
δx = (x[2] - x[1]) / 2
δy = (y[2] - y[1]) / 2
cbarPal = :Spectral_11
ztmp = (z .- minimum(z)) ./ (maximum(z .- minimum(z)))
cmap = colormap("RdBu", 15*15)
cmap2 = reshape(cmap, size(z))
ztmp2 = abs.(z) ./ maximum(abs.(z)) .+ 0.15

function histogram_or_bars_in_3d()
    fig = Figure(size=(1200, 800), fontsize=26)
    ax1 = Axis3(fig[1, 1]; aspect=(1,1,1), elevation=π/6, perspectiveness=0.5)
    ax2 = Axis3(fig[1, 2]; aspect=(1,1,1), perspectiveness=0.5)
    rectMesh = Rect3f(Vec3f(-0.5, -0.5, 0), Vec3f(1, 1, 1))
    meshscatter!(ax1, x, y, 0 * z; marker=rectMesh, color=z[:],
        markersize=Vec3f.(2δx, 2δy, z[:]), colormap=:Spectral_11,
        shading=NoShading)
    limits!(ax1, -3.5, 3.5, -3.5, 3.5, -7.45, 7.45)
    meshscatter!(ax2, x, y, 0 * z; marker=rectMesh, color=z[:],
        markersize=Vec3f.(2δx, 2δy, z[:]), colormap=(:Spectral_11, 0.25),
        shading=NoShading, transparency=true)
    for (idx, i) in enumerate(x), (idy, j) in enumerate(y)
        rectMesh=Rect3f(Vec3f(i-δx, j-δy, 0), Vec3f(2δx, 2δy, z[idx,idy]))
        recmesh=GeometryBasics.mesh(rectMesh)
        lines!(ax2, recmesh; color=(cmap2[idx, idy], ztmp2[idx, idy]))
    end
    fig
end
histogram_or_bars_in_3d()

In [17]:
using GLMakie, LinearAlgebra, Colors
function filled_line_and_linesegments_in_3D()
    xs = range(-3, 3, 50)
    lower = [Point3f(i, -i, 0) for i in range(0, 3, 100)]
    upper = [Point3f(i, -i, sin(i) * exp(-(i + i)))
        for i in range(0, 3, length=100)]
    fig = Figure(size=(1200, 800))
    axs = [Axis3(fig[1, i]; elevation=π/6, perspectiveness=0.5) for i=1:2]
    band!(axs[1], lower, upper; color=repeat(norm.(upper), outer=2),
        colormap=:CMRmap)
    lines!(axs[1], upper, color=:black)
    # linesegments!(axs[2], cos.(xs), xs, sin.(xs); linewidth=5,color=1:length(xs))
    colors = [RGBA(rand(), rand(), rand(), 0.5) for i in 1:50]
    println("colors = ", colors)
    lines!(axs[2], cos.(xs), xs, sin.(xs); linewidth=5, color=colors)
    fig
end
filled_line_and_linesegments_in_3D()

colors = RGBA{Float64}[RGBA{Float64}(0.27544665620472697,0.41807447851007007,0.11522136341312839,0.5), RGBA{Float64}(0.5713292531715959,0.030971611796269616,0.9027357452251894,0.5), RGBA{Float64}(0.842589950863431,0.06350451772306043,0.6264110947396896,0.5), RGBA{Float64}(0.47486035527640846,0.13172548885240576,0.9753579720773797,0.5), RGBA{Float64}(0.8513225673843184,0.5187475698698162,0.2842581068673138,0.5), RGBA{Float64}(0.18654379390933018,0.7251296074607243,0.05564606433821806,0.5), RGBA{Float64}(0.8589857315207474,0.6028087795094458,0.3884241295059677,0.5), RGBA{Float64}(0.9894341560657139,0.19188306897789487,0.34841255426897366,0.5), RGBA{Float64}(0.6571403530633555,0.4144561053359116,0.4965468160425869,0.5), RGBA{Float64}(0.8353732660315243,0.47537067062898164,0.6751617497200206,0.5), RGBA{Float64}(0.08225866085727129,0.4498359485689004,0.8882060694472589,0.5), RGBA{Float64}(0.9959065290370194,0.7785709883549864,0.9740585088760577,0.5), RGBA{Float64}(0.1446957054303858,0.87785

### Animations

In [18]:
using GLMakie

time = Observable(0.0)

xs = range(0, 7, length=40)

ys_1 = @lift(sin.(xs .- $time))
ys_2 = @lift(cos.(xs .- $time) .+ 3)

fig = lines(xs, ys_1, color = :blue, linewidth = 4,
    axis = (title = @lift("t = $(round($time, digits = 1))"),))
scatter!(xs, ys_2, color = :red, markersize = 15)
display(fig)

framerate = 30
timestamps = range(0, 2, step=1/framerate)

for t in timestamps
    time[] = t
    sleep(1/framerate)
end

In [19]:
# # Making animated and interactive scientific 
# # visualizations in Makie.jl

# Alrighty, in this short tutorial I'll teach you how to make
# animated scientific such as this video, or this application.

# Prerequisitives:
# * Basic familiarity with Julia (obviously)
# * Basic familiarity with Makie.jl

# # The process
# 0. Learn about `Observable`s
# 1. Initialize simulation in a stepping manner
# 2. Initialize the `Observable`s of the animation 
# 3. Plot the `Observable`s and any other static elements
# 4. Create the "animation stepping function"
# 5. Test it
# 6. Save animations to videos
# 7. Interactive application

# Let's start!
# Load packages that will be used
using DynamicalSystems
using OrdinaryDiffEq
using GLMakie
using DataStructures: CircularBuffer

# %% 0. Learn about `Observable`s
# An `Observable` is a mutable container of an object of type `T`.
# `T` can be any type. The value of an `Observable` can then be 
# changed on the spot, just like updating any mutable container.
# (This is similar to the base structure `Ref`, if you're familiar)

# The important part here is that `Observable`s can be "listened" to.
# What does this mean...?

o = Observable(1) # Observable with values of type `Int`

l1 = on(o) do val # Create a listener `l1` of observable.
    println("Observable now has value $val")
end

# `l1` is triggered each time the value of `o` is updated.
# (demo in REPL, set `o[] = 2`.)

# We care about `Observable`s because Makie.jl is hooked up
# to this "listener" system. If any plotted element is
# initialized as an observable, then Makie.jl understands this.
# Updating the observable would update the plot values.

# For example:
ox = 1:4
oy = Observable(rand(4))
lw = Observable(2)

fig, ax = lines(ox, oy; linewidth = lw)
ylims!(ax, 0, 1)

lw[] = 50
oy[] = rand(4)

# This simple process is the basis of creating 
# both animations, as well as interactive apps with Makie.jl.
# So in the following let's begin making a simple visualization
# and interactive application of the double pendulum


# %% 1. Initialize simulation in a stepping manner
# (this can also be done with a pre-run simulation)
# the goal is to create a "step" function which
# once called it progresses the data for one animation frame
const L1 = 1.0
const L2 = 0.9
M = 2
u0 = [π/3, 0, 3π/4, -2]
dp = Systems.double_pendulum(u0; L1, L2)

# Solve diffeq with constant step for smoother curves
diffeq = (alg = Tsit5(), adaptive = false, dt = 0.005)

integ = dp.integ

function xycoords(state)
    θ1 = state[1]
    θ2 = state[3]
    x1 = L1 * sin(θ1)
    y1 = -L1 * cos(θ1)
    x2 = x1 + L2 * sin(θ2)
    y2 = y1 - L2 * cos(θ2)
    return x1,x2,y1,y2
end

# `integ` is an integrator. `step!(integ)` progresses the integrator
# for one step. `integ.u` is the system state at current step. 
# Then `xycoords` converts the states of the integrator
# to their plottable format. So we can imagine something like 
function progress_for_one_step!(integ)
    step!(integ)
    return xycoords(integ)
end
# to be our stepping function that returns the new data.

# If we had finite data instead of a forever-running animation, 
# then the "stepping function" would simply be to progress the index `i`
# of the existing data one step forwards...


# %% 2. Initialize the `Observable`s of the animation 
# You need to think of this in advance: what things will to be 
# animated, and what other plotting elements will be static? 
# Animated elements will need to become `Observable`s.

# Here the animated elements will be: balls and rods making the
# double pendulum, and the tail (trajectory) of the pendulum.
x1,x2,y1,y2 = xycoords(u0)
rod   = Observable([Point2f(0, 0), Point2f(x1, y1), Point2f(x2, y2)])
balls = Observable([Point2f(x1, y1), Point2f(x2, y2)])
# (Remember: the most optimal way to plot 2D things in Makie.jl is to
# give it a vector of `Point2f`, the coordinates for the plot)

# Here we have initialized two _different_ observables, because
# rods and balls will be plotted in a different manner (lines/scatter)

# Next is the observable for the tail
tail = 300 # length of plotted trajectory, in units of `dt`
# The circular buffer datastructure makes making stepping-based
# animations very intuitive
traj = CircularBuffer{Point2f}(tail)
fill!(traj, Point2f(x2, y2)) # add correct values to the circular buffer
traj = Observable(traj) # make it an observable


# %% 3. Plot the `Observable`s and any other static elements
# Before plotting we need to initialie a figure
fig = Figure(); display(fig)
# in my experience it leads to cleaner code if we first initialize 
# an axis and populate it accordingly.
ax = Axis(fig[1,1])

# Now we plot the observables _directly_! First the pendulum
lines!(ax, rod; linewidth = 4, color = :purple)
scatter!(ax, balls; marker = :circle, strokewidth = 2, 
    strokecolor = :purple,
    color = :black, markersize = [8, 12]
)

# then its trajectory, with a nice fadeout color
c = to_color(:purple)
tailcol = [RGBAf(c.r, c.g, c.b, (i/tail)^2) for i in 1:tail]
lines!(ax, traj; linewidth = 3, color = tailcol)

# We can also plot now any other static elements
ax.title = "double pendulum"
ax.aspect = DataAspect()
l = 1.05(L1+L2)
xlims!(ax, -l, l)
ylims!(ax, -l, 0.5l)

# %% 4. Create the "animation stepping function"
# Using the functions of step 1, we now define a function
# that updates the observables. Makie.jl understands observable
# updates and directly reflects this on the plotted elements.
function animstep!(integ, rod, balls, traj)
    x1,x2,y1,y2 = progress_for_one_step!(integ)
    rod[] = [Point2f(0, 0), Point2f(x1, y1), Point2f(x2, y2)]
    balls[] = [Point2f(x1, y1), Point2f(x2, y2)]
    push!(traj[], Point2f(x2, y2))
    traj[] = traj[] # <- important! Updating in-place the value of an
                    # `Observable` does not trigger an update!
end

# %% 5. Test it
for i in 1:1000
    animstep!(integ, rod, balls, traj)
    sleep(0.005)
end


In [20]:

# cool it works. Let's wrap up the creation of the observables
# and plots in a function (just to re-initialie everything)
function makefig(u0)
    dp = Systems.double_pendulum(u0; L1, L2)
    integ = dp.integ
    x1,x2,y1,y2 = xycoords(u0)
    rod   = Observable([Point2f(0, 0), Point2f(x1, y1), Point2f(x2, y2)])
    balls = Observable([Point2f(x1, y1), Point2f(x2, y2)])
    traj = CircularBuffer{Point2f}(tail)
    fill!(traj, Point2f(x2, y2)) # add correct values to the circular buffer
    traj = Observable(traj) # make it an observable
    fig = Figure(); display(fig)
    ax = Axis(fig[1,1])
    lines!(ax, rod; linewidth = 4, color = :purple)
    scatter!(ax, balls; marker = :circle, strokewidth = 2, 
        strokecolor = :purple,
        color = :black, markersize = [8, 12]
    )
    lines!(ax, traj; linewidth = 3, color = tailcol)
    ax.title = "double pendulum"
    ax.aspect = DataAspect()
    l = 1.05(L1+L2)
    xlims!(ax, -l, l)
    ylims!(ax, -l, 0.5l)
    # also return the figure object, we'll ues it!
    return fig, integ, rod, balls, traj
end
    
# %% 6. Save animations to videos
fig, integ, rod, balls, traj = makefig(u0)
frames = 1:200
record(fig, "video.mp4", frames; framerate = 60) do i # i = frame number
    for j in 1:5 # step 5 times per frame
        animstep!(integ, rod, balls, traj)
    end
    # any other manipulation of the figure here...
end # for each step of this loop, a frame is recorded

# %% 7. Interactive application
# Makie.jl has tremendously strong capabilities for real-time
# interactivity. To learn all of this takes time of course,
# and you'll need to consult the online documentation.
# Here we will do two interactions: 1) a play/stop button
# 2) clicking on the screen and getting a new initial condition!

fig, integ, rod, balls, traj = makefig(u0)
# The run button is actually pretty simple, we'll add it below the plot
run = Button(fig[2,1]; label = "run", tellwidth = false)
# This button will start/stop an animation. It's actually surprisingly
# simple to do this. The magic code is:
isrunning = Observable(false)
on(run.clicks) do clicks; isrunning[] = !isrunning[]; end
on(run.clicks) do clicks
    @async while isrunning[]
        isopen(fig.scene) || break # ensures computations stop if closed window
        animstep!(integ, rod, balls, traj)
        sleep(0.01) # or `yield()` instead
    end
end

# `on` an important Observables function when one starts
# doing advanced stuff. It triggers a piece of code once an observable
# triggers its update.

# We'll add one more interactive feature which will trigger once
# we click on the axis. Notice that by default makie performs a zoom
# once one clicks on the axis, so we'll disable this
ax = content(fig[1,1])
Makie.deactivate_interaction!(ax, :rectanglezoom)
# and we'll add a new trigger using the `select_point` function:
spoint = select_point(ax.scene, marker = :circle)

# Now we see that we can click on the screen and the `spoint` updates!

# Okay, let's do something useful when it triggers
function θωcoords(x, y)
    θ = atan(y,x) + π/2
    return SVector(θ,0,0,0)
end

on(spoint) do z
    x, y = z
    u = θωcoords(x, y)
    reinit!(integ, u)
    # Reset tail and balls to new coordinates
    x1,x2,y1,y2 = xycoords(u)
    traj[] .= fill(Point2f(x2, y2), length(traj[]))
    traj[] = traj[]
    rod[] = [Point2f(0, 0), Point2f(x1, y1), Point2f(x2, y2)]
    balls[] = [Point2f(x1, y1), Point2f(x2, y2)]
end

ObserverFunction defined at /Users/mg/repos/ez_mag_sim/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X42sZmlsZQ==.jl:83 operating on Observable(Float32[0.0, 0.0])